In [17]:
#필수 모듈 불러오기
import cv2
import mediapipe as mp
import pyglet
import serial

#변수 설정
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

#카메라객체생성
cap = cv2.VideoCapture(0)

#현재 카메라 해상도 얻기
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)

#시리얼 통신 객체 생성
ser = serial.Serial('COM14', 9600)

#음원재생,시리얼통신함수
def play(facein,faceout):
    #얼굴이들어오면
    if facein:
        #로드,재생,아두이노에명령보내기
        source1=pyglet.media.load("welcome.mp3")
        source1.play()
        ser.write(b'h')
        return
    #얼굴이나가면
    if faceout:
        source2=pyglet.media.load("goodbye.mp3")
        source2.play()
        ser.write(b'h')
        return
    
#음원 재생 제어변수
facein=True
faceout=True

#모델파일열기
with mp_face_detection.FaceDetection() as face_detection:
    
    while True:
        
        #카메라읽기
        success, image = cap.read()
        
        #전처리
        image = cv2.flip(image, 1)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        #모델 예측, 얼굴 위치, 특징점 위치 반환
        results = face_detection.process(image)
        
        #보기좋게변환
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        #랜드마크가 있으면==얼굴이 인식되면
        if results.detections:
            #모든 랜드마크에 대해
            for detection in results.detections:
                
                #원 좌표 계산
                xmin=detection.location_data.relative_bounding_box.xmin
                ymin=detection.location_data.relative_bounding_box.ymin
                boxw=detection.location_data.relative_bounding_box.width
                boxh=detection.location_data.relative_bounding_box.height
                cx=int((xmin+0.5*boxw)*width)
                cy=int((ymin+0.5*boxh)*height)
                r=int(0.5*boxw*width)+20
                
                #원 그리기
                cv2.circle(image,(cx,cy-20),r,(255,255,255),2)
                #텍스트 붙이기
                cv2.putText(image,"Face",(cx-35,cy-r-30),cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,0),2)
                
                #얼굴이들어오면
                faceout=False
                #재생
                play(facein,faceout)
                #반복재생차단
                facein, faceout = False, True
            
        #얼굴인식이안될때
        else:
            #얼굴이나가면
            facein=False
            #재생
            play(facein,faceout)
            #반복재생차단
            facein, faceout = True , False
            
        #창띄우기
        cv2.imshow('image', image)

        
        #q눌러빠져나오기
        if cv2.waitKey(1) == ord('q'):
            break
    
    #사용한 카메라, 창 닫기
    ser.close()
    cap.release()
    cv2.destroyAllWindows()

In [6]:
"""
detection 클래스 구성 멤버 예시
label_id: 0
score: 0.9364964365959167
location_data {
  format: RELATIVE_BOUNDING_BOX
  relative_bounding_box {
    xmin: 0.2917987108230591
    ymin: 0.4627208113670349
    width: 0.397702693939209
    height: 0.5302088260650635
  }
  relative_keypoints {
    x: 0.4000580608844757
    y: 0.5988087058067322
  }
  relative_keypoints {
    x: 0.5627859234809875
    y: 0.6001312136650085
  }
  relative_keypoints {
    x: 0.4735056161880493
    y: 0.7241043448448181
  }
  relative_keypoints {
    x: 0.4762313961982727
    y: 0.838823139667511
  }
  relative_keypoints {
    x: 0.32249829173088074
    y: 0.6547989845275879
  }
  relative_keypoints {
    x: 0.6643185615539551
    y: 0.6592555046081543
  }
}

접근 방법예시
detection.location_data.relative_bounding_box.xmin
"""

'\ndetection 클래스 구성 멤버 예시\nlabel_id: 0\nscore: 0.9364964365959167\nlocation_data {\n  format: RELATIVE_BOUNDING_BOX\n  relative_bounding_box {\n    xmin: 0.2917987108230591\n    ymin: 0.4627208113670349\n    width: 0.397702693939209\n    height: 0.5302088260650635\n  }\n  relative_keypoints {\n    x: 0.4000580608844757\n    y: 0.5988087058067322\n  }\n  relative_keypoints {\n    x: 0.5627859234809875\n    y: 0.6001312136650085\n  }\n  relative_keypoints {\n    x: 0.4735056161880493\n    y: 0.7241043448448181\n  }\n  relative_keypoints {\n    x: 0.4762313961982727\n    y: 0.838823139667511\n  }\n  relative_keypoints {\n    x: 0.32249829173088074\n    y: 0.6547989845275879\n  }\n  relative_keypoints {\n    x: 0.6643185615539551\n    y: 0.6592555046081543\n  }\n}\n\n접근 방법예시\ndetection.location_data.relative_bounding_box.xmin\n'